In [60]:
import re
import pandas as pd
from g4f.client import Client
from g4f.Provider import RetryProvider, Phind, FreeChatgpt, Liaobots
import g4f.debug
import nest_asyncio
nest_asyncio.apply()
g4f.debug.logging = True

In [61]:
df = pd.read_csv("data/Sustainability Courses Offered - 1 - Course Review.csv")
df.head()

,Course,Course URL,Subject,Last Offered,Course Title,Credits Offered,Assessment of Sustainability Focused or Inclusive,Offered to UG/G/Both or Other,Justification for classification (including quotes from syllabi/course descriptions where applicable),Description,Course Home Department,Last Catalog Year Reviewed,Unnamed: 12,"The courses were counted using catalog listings with language that is aligned with the Earth Charter, covering topics such as ecological integrity, social and economic justice, and democracy and nonviolence. Course descriptions that included terms like ""environment,"" ""sustainable/sustainability,"" and ""energy"" were counted as Sustainability-Focused courses, and the other courses that address a topic from the Earth Charter but are not centrally-focused on sustainability were considered Sustainability-Included courses.",Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,AAAS,https://courses.rice.edu/courses/courses/!SWKS...,200,Fall 2021,Knowing Blackness: Introduction To African And...,3,N,UG,Has a strong cultural emphasis of how the Afri...,An exploration of the origins and development ...,African And African American Studies,2021-2022,NaN,NaN,NaN,NaN,NaN,Percentage of Sustainability Course Offerings
1,AAAS,https://courses.rice.edu/courses/courses/!SWKS...,300,Spring 2022,Writing Black Lives,3,Y - Inclusive,UG,This course focuses on Black Writers and their...,"In this course, students will be reading, refl...",African And African American Studies,2021-2022,NaN,NaN,NaN,NaN,NaN,27.94%
2,AAAS,NaN,510,Spring 2022,Intro To Diasporic Studies,3,N,G,This course is for graduate students who are i...,This is the core course for the Certificate in...,African And African American Studies,2021-2022,NaN,NaN,NaN,NaN,NaN,NaN
3,AAAS,NaN,600,Fall 2021,African And African American Studies Colloquium,3,N,G,This course is for graduate students who are i...,Through readings and discussions the colloquiu...,African And African American Studies,2021-2022,NaN,NaN,NaN,NaN,NaN,NaN
4,AFSC,https://courses.rice.edu/courses/courses/!SWKS...,101,Fall 2021,Foundation Of The Usaf I,2,N,UG,No indication of sustainability focus or inclu...,Overall roles and missions of the USAF; career...,Air Force Science,2021-2022,NaN,NaN,NaN,NaN,NaN,NaN


<h1>GPT Stuff </h1>

In [62]:
def create_client():
    provider=RetryProvider([FreeChatgpt], shuffle=False)
    client = Client(
        provider=provider
    )

    return client, provider

In [63]:
def generate_prompts(descriptions):
    prompts = []
    for i in descriptions:
        prompt = f"""A course description is provided below:\n\n{i}\n\nBased on this course description does this class promote sustainability? 
                    Please assess it with a 1 (sustainable) or 0 (not sustainable) and provide a one sentence explanation why.
                    The format should be 1 or 0: (Reasoning) (newline)"""
        prompts.append(prompt)

    return prompts

In [64]:
def run_model(client, provider, prompt):
    response = client.chat.completions.create(
        model="gemini",
        messages=[{"role": "user", "content": f"{prompt}"}],
        provider=provider
    )

    return response

In [65]:
def parse_response(response):
    eval = []
    reason = []
    for i, j in [string.split(': ', 1) for string in response.choices[0].message.content.split('\n')]:
        eval.append(i)
        reason.append(j)
    
    return eval, reason

In [66]:
client, provider = create_client()

In [67]:
print(generate_prompts(df.Description))

['A course description is provided below:\n\nAn exploration of the origins and development of African Studies and African American Studies. Through a focus on the articulation and resolution of field-changing debates, the course introduces students to methodologies and practices that have led to and that continue to lead to knowing Africa and African-descendent people with earnest regard for the complexity and subtlety that the subjects require.\n\nBased on this course description does this class promote sustainability? \n                    Please assess it with a 1 (sustainable) or 0 (not sustainable) and provide a one sentence explanation why.\n                    The format should be 1 or 0: (Reasoning) (newline)', 'A course description is provided below:\n\nIn this course, students will be reading, reflecting, and dissecting short stories, novels, television scripts, and other works of fiction crafted by artists across the Black diaspora. Students will thoroughly discuss process a

In [70]:
def run(client, provider, descriptions):
    count = 100
    evals, reasons = [], []
    while count < len(descriptions):
        eval, reason = parse_response(run_model(client, provider, descriptions[(count-100):count]))
        evals += eval
        reasons += reason
        count += 100
    eval, reason = parse_response(run_model(client, provider, descriptions[(count-100):]))
    evals += eval
    reasons += reason

    return evals, reasons

In [71]:
evals, reasons = run(*create_client(), generate_prompts(df.Description))

Using RetryProvider provider and gemini model
Using FreeChatgpt provider
Using RetryProvider provider and gemini model
Using FreeChatgpt provider
FreeChatgpt: ClientResponseError: 429, message='Too Many Requests', url=URL('https://free.chatgpt.org.uk/api/openai/v1/chat/completions')


RetryProviderError: RetryProvider failed:
FreeChatgpt: ClientResponseError: 429, message='Too Many Requests', url=URL('https://free.chatgpt.org.uk/api/openai/v1/chat/completions')

In [ ]:
print(evals)
print(reasons)

['1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '1', '0', '0', '0', '1', '0', '0', '1', '0', '0', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',

In [ ]:
print(sum([int(i) for i in evals]))

37


<h1> Keywords </h1>

In [81]:
def read_keyword_txt(path):
    keywords = set()
    with open(path, "r") as file:
        line = file.readline()
        while line:
            keywords.add(line.strip("\n"))
            line = file.readline()

    return keywords

In [82]:
def check_keywords(keywords, descriptions):
    # Convert keywords list to a set for faster lookup
    results = []
    for description in descriptions:
        # Split description into words
        if not pd.isna(description):
            words = description.split()
            # Check if any word in the description is in the set of keywords
            if any(word in keywords for word in words):
                results.append("Y")
            else:
                results.append("N")
        else:
            results.append(False)
    return results

In [91]:
print(read_keyword_txt("words.txt"))

{'Society', 'Soil', 'Discrimination', 'Poverty', 'Culture', 'Deforestation', 'Resource Management', 'Justice', 'Conservation', 'Biomes', 'Equality', 'Ocean', 'Efficiency', 'Wetlands', 'GHG', 'Sustainability', 'CO2', 'Organic', 'Restoration', 'Energy', 'Globalization', 'Recycling', 'Equity', 'Wilderness', 'Environmental Policy', 'Consumption', 'Earth', 'Biodiversity', 'Emission', 'Global', 'Hydrology', 'Ecology', 'Habitat', 'Green', 'Natural', 'Recycle', 'Social', 'Tree', 'Waste', 'Wildlands', 'Public Safety', 'Air', 'Nutrition', 'Preservation', 'Pollution', 'Community', 'Natural Resources', 'Racism', 'Nature', 'Invasive Species', 'Wildlife', 'Alternative Transportation', 'Ethnicity', 'Environment', 'Vegetation', 'Carbon', 'Disparity', 'Ecotourism', 'Erosion', 'Climatechange', 'Ecosystem', 'Land Use', 'Geography', 'Race', 'Rivers', 'Sustainable', 'Hydrologic', 'Water', 'Renewable', 'Environmental', 'Integrated Pest Management'}


In [92]:
total = check_keywords(read_keyword_txt("words.txt"), df.Description)

In [93]:
print(sum(total))
print(len(total))

95
6492
